# Organização da Pipeline de pré-processamento

A definição dos tratamentos e transformações dos dados foi realizada com base nos insights extraídos durante a análise exploratória (EDA). Para otimizar essa etapa, foi utilizada uma [planilha de organização](https://docs.google.com/spreadsheets/d/12IVVANl1VLA55f7QB15Y_hmrT_lGEXov8IsCHVvneR0/edit?usp=sharing), que ajudou a planejar os tratamentos e transformações necessários em cada variável.

Com o objetivo de melhorar a reprodutibilidade e escalabilidade do processo, foram desenvolvidas duas pipelines separadas: Pipeline de Tratamento e Pipeline de Transformação. Essas pipelines foram aplicadas separadamente no conjunto de treino e de teste para evitar o risco de data leakage.

### Pipeline de tratamento:

Realizada para sanar problemas de ausência de engenharia de dados. Nessa pipeline, foi buscado resolver os seguintes problemas:

* Remoção de colunas desconsideradas para o modelo
* Remover underscore e espaços vazios
* Transformar dados nos tipos corretos
* Tratar outliers
* Feature engineering: converter a variável categórica `Credit_History_Age` (tempo total de histórico de crédito do cliente) no total de meses.

### Pipeline de transformação:

Realizada para preparar os dados para os diferentes modelos de machine learning.

* Dados numéricos: substituir dados ausentes pela média
* Dados categóricos: substituir ausentes por "missing"
* Dados categóricos (encoder): passar um encoder (ex: Target Encoder)
* Todas as variáveis (scale): passar um scaler (ex: MinMax ou Robust Scaler)

# Bibliotecas

In [57]:
# Data manipulation

import pandas as pd
import numpy as np

# Data viz

import matplotlib.pyplot as plt
import seaborn as sns

# 1. Importação dos dados

In [2]:
train = pd.read_csv('data/train.csv', low_memory=False)
test = pd.read_csv('data/test.csv', low_memory=False)

In [5]:
# Dados de treino

train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [10]:
# Dimensionalidade (dados de treino)

train.shape

(100000, 28)

In [6]:
# Dados de teste

test.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [11]:
# Dimensionalidade (dados de teste)

test.shape

(50000, 27)

**OBS (1):** nos dados de teste há uma variável a menos, que é a 'Credit_Score', inicialmente definida como o target.  
**OBS (2):** o alvo da base de dados era a variável Credit_Score, que consiste na pontuação de crédito do cliente. Nessa classificação, haviam indivíduos classificados como: Poor, Standard e Good. Para deixar o problema ainda mais próximo do proposto na concessão de crédito, optou-se por modificar as classes, como indicado abaixo.  

* Clientes classificados como `Poor` ➡ `Classe 1` (maus pagadores)  
* Clientes classifiados como `Standard` ➡ `Classe 0` (bons pagadores)  
* Clientes classifiados como `Good` ➡ `Classe 0` (bons pagadores)

**OBS (3):** o objetivo original da base de dados era prever a variável Credit_Score, que reflete a pontuação de crédito do cliente, com base em variáveis como comportamento de pagamento e dívida pendente. No entanto, o foco foi ajustado para classificar futuros clientes em bons ou maus pagadores, visando prevenir inadimplências durante o processo de concessão de crédito. As variáveis relacionadas ao comportamento dos clientes, que já constavam na base de dados, foram consideradas como informações complementares, hipoteticamente fornecidas por outro birô de crédito (ex: Serasa, SPC, etc).

# 2. Pipeline de tratamento

Por motivos de falta de informações úteis ao aprendizado do modelo, bem como a possíveis vazamentos de dados, foram desconsideradas as variáveis: 
* Customer_ID
* SSN
* ID
* Name
* Delay_from_due_date
* Num_of_Delayed_Payment

Todo o restante das variáveis foi inicialmente mantido, para ser avaliado posteriormente na etapa de feature selection.

A pipeline de tratamento seguiu com:
1) 

In [38]:
# Dropar colunas desconsideradas para o modelo

def drop_variaveis(df):
    lista_variaveis = ['ID', 'Customer_ID', 'SSN', 'Name', 'Delay_from_due_date', 'Num_of_Delayed_Payment']
    return df.drop(columns=lista_variaveis, axis=1)


#############################################################################################################

# Exclusivas para algumas variáveis

# Transformar dados nos tipos corretos

def clean_columns_and_convert(df):
    # Limpar os nomes das colunas
    df.columns = df.columns.str.replace('_', '').str.strip()
    
    # Substituir valores vazios ou espaços em branco por NaN
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    
    # Separando variáveis numéricas
    lista_var_numericas_int = ['Age', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
                               'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries']
    
    lista_var_numericas_float = ['Annual_Income', 'Monthly_Inhand_Salary', 'Changed_Credit_Limit', 'Outstanding_Debt', 
                                 'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']
    
    # Removendo linhas com NaN nas variáveis numéricas
    #df.dropna(subset=lista_var_numericas_int + lista_var_numericas_float, inplace=True)
    
    # Transformando variáveis numéricas inteiras
    df[lista_var_numericas_int] = df[lista_var_numericas_int].replace(r'[^0-9]', '', regex=True).astype(int)
    
    # Transformando variáveis numéricas float
    df[lista_var_numericas_float] = df[lista_var_numericas_float].replace(r'[^0-9.]', '', regex=True)
    
    # Removendo entradas vazias e convertendo para float
    df[lista_var_numericas_float] = df[lista_var_numericas_float].replace('', np.nan).astype(float)
    
    return df

# Tratar categorias inválidas e substituir por nans

def nan_transform(df):
    # Definindo as categorias inválidas para cada coluna
    categorias_invalidas = {
        'Payment_of_Min_Amount': 'NM',
        'Payment_Behaviour': '!@9#%8',
        'Credit_Mix': '_'
    }

    # Substituindo as categorias inválidas por NaN para as colunas específicas
    for coluna, categoria_invalida in categorias_invalidas.items():
        df[coluna] = df[coluna].replace(categoria_invalida, np.nan)
    
    return df

# Exclusiva para a variável Credit_History_Age: vai se transformat no total de meses
# FEATURE ENGINEERING
def convert_to_months(age):
    """Função para converter Years and months em meses"""

    if isinstance(age, str):
        try:
            years, months = age.split(' Years and ')
            return int(years) * 12 + int(months.split(' ')[0])
        except ValueError:
            return None  # Retorna None se o formato estiver incorreto
    else:
        return None

In [58]:
# Variaveis com outliers (acrescentar na pipeline de tratamento): Age, Num_Bank_Accounts, Num_Credit_Card (clip:pandas)

In [41]:
# Testar tudo acima

In [53]:
# Passar tudo pro formato abaixo

In [49]:
from sklearn.base import BaseEstimator, TransformerMixin

class DropVariaveis(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        lista_variaveis = ['ID', 'Customer_ID', 'SSN', 'Name', 'Delay_from_due_date', 'Num_of_Delayed_Payment']
        return X.drop(columns=lista_variaveis, axis=1)

class CleanColumnsAndConvert(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X.columns = X.columns.str.replace('_', '').str.strip()
        X.replace(r'^\s*$', np.nan, regex=True, inplace=True)

        lista_var_numericas_int = ['Age', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
                                   'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries']
        lista_var_numericas_float = ['Annual_Income', 'Monthly_Inhand_Salary', 'Changed_Credit_Limit', 'Outstanding_Debt',
                                     'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']

        X.dropna(subset=lista_var_numericas_int + lista_var_numericas_float, inplace=True)

        X[lista_var_numericas_int] = X[lista_var_numericas_int].replace(r'[^0-9]', '', regex=True).astype(int)
        X[lista_var_numericas_float] = X[lista_var_numericas_float].replace(r'[^0-9.]', '', regex=True)
        X[lista_var_numericas_float] = X[lista_var_numericas_float].replace('', np.nan).astype(float)
        return X

class NanTransform(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        categorias_invalidas = {
            'Payment_of_Min_Amount': 'NM',
            'Payment_Behaviour': '!@9#%8',
            'Credit_Mix': '_'
        }

        for coluna, categoria_invalida in categorias_invalidas.items():
            X[coluna] = X[coluna].replace(categoria_invalida, np.nan)
        return X

class ConvertToMonths(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['Credit_History_Age'] = X['Credit_History_Age'].apply(self.convert_to_months)
        return X

    def convert_to_months(self, age):
        if isinstance(age, str):
            try:
                years, months = age.split(' Years and ')
                return int(years) * 12 + int(months.split(' ')[0])
            except ValueError:
                return None
        else:
            return None

In [46]:
# Construção da pipeline

from sklearn.pipeline import Pipeline

# Criando a pipeline com os passos de transformação
pipeline = Pipeline(steps=[
    ('drop_variaveis', DropVariaveis()),
    ('clean_columns_and_convert', CleanColumnsAndConvert()),
    ('nan_transform', NanTransform()),
    ('convert_to_months', ConvertToMonths())
])

In [47]:
# Aplicando a pipeline aos dados de treino e teste
#train_transformado = pipeline.fit_transform(train)
#test_transformado = pipeline.transform(test)

# 2. Pipeline de transformação

In [54]:
# Scalers e encoders